# Multiple Imputation with Chain Equations (MICE) Algorithm

### Notes

Choosing appropriate imputation methods is crucial to maintaining consistent data distributions. Missing data often occurs in independent variables for various reasons, such as, human error, data entry problems, or equipment malfunctions during data collection.

Most of machine learning models expect the data to be complete without any trace of null values. Rows with missing data points can detrimentally affect the models’ predictive power, introducing bias, reducing the model’s accuracy and making some statistical analyses inapplicable. This is why it is essential to tackle missing data effectively.

The act of replacing missing values by estimates, is called missing data imputation.

MICE (Multiple Imputation by Chained Equations) is a powerful technique for handling missing data in datasets. Instead of simply deleting rows with missing values or replacing them with a single estimate (like the mean), MICE creates multiple plausible versions of the complete dataset, reflecting the uncertainty around the missing values.

In [52]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, SimpleImputer, IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler


In [53]:
#data = pd.read_csv('https://raw.githubusercontent.com/Impact-Insights/Data-Engineering-Projects/refs/heads/main/Customer%20Purchases%20Project/Customer_Purchase_Data.csv?token=GHSAT0AAAAAAC7OUMXN6GPU7MWDK5RBLW3GZ6G5I4Q')
data = pd.read_csv(r'Customer_Purchase_Data.csv')
data.head()

,Unnamed: 0,Customer_ID,Age,Salary,City,Gender,Purchase_Amount,Signup_Date,Marital_Status,Education
0,0,1,40.0,70000.0,New York,NaN,619,2022-01-01,Widowed,PhD
1,1,2,45.0,30000.0,Houston,Female,1945,2022-01-02,Single,PhD
2,2,3,35.0,30000.0,New York,Male,2783,2022-01-03,NaN,Master
3,3,4,45.0,30000.0,NaN,Other,3913,2022-01-04,Married,Bachelor
4,4,5,45.0,30000.0,New York,Male,4771,2022-01-05,Married,NaN


In [54]:
data = data.drop(columns = ['Unnamed: 0'])

## Cleaning and Encoding values in the Gender Column

### Extracting Data to work on.

In [55]:
g_data = data[['Age', 'Salary', 'Purchase_Amount', 'Gender']]
g_data.head()

,Age,Salary,Purchase_Amount,Gender
0,40.0,70000.0,619,NaN
1,45.0,30000.0,1945,Female
2,35.0,30000.0,2783,Male
3,45.0,30000.0,3913,Other
4,45.0,30000.0,4771,Male


### Standardizing and Normalizing the data in categorical columns.

In [56]:
g_data['Gender'].unique()

array([nan, 'Female', 'Male', 'Other', 'male'], dtype=object)

In [57]:
g_data.loc[:, 'Gender'] = g_data.loc[:, 'Gender'].replace('male', 'Male')
g_data['Gender'].unique()

array([nan, 'Female', 'Male', 'Other'], dtype=object)

In [58]:
#creating columns for encoding
impute_col = 'Gender'
impute_col_encoded = 'Gender_Encoded'

### Encoding the data into numerical representation for the categories.

In [59]:
le = LabelEncoder()

Fitting the data into the model

In [60]:
le.fit(g_data[impute_col])

LabelEncoder()

Transforming our categories into numerical classes, getting the results in a dictionary.

In [61]:
dict(zip(le.classes_, le.transform(le.classes_)))

{'Female': np.int64(0),
 'Male': np.int64(1),
 'Other': np.int64(2),
 nan: np.int64(3)}

### Applying Label Transformation
Creating the numeric form of the 'categories' into the `Gender_Encoded` column

In [62]:
g_data[impute_col_encoded] = le.transform(g_data[impute_col])
#g_data[['Age', 'Salary', 'Purchase_Amount', 'Gender', 'Gender_Encoded']].head()

C:\Users\CAPACITI-JHB\AppData\Local\Temp\ipykernel_4816\2240981684.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g_data[impute_col_encoded] = le.transform(g_data[impute_col])


In [63]:
g_data.head()

,Age,Salary,Purchase_Amount,Gender,Gender_Encoded
0,40.0,70000.0,619,NaN,3
1,45.0,30000.0,1945,Female,0
2,35.0,30000.0,2783,Male,1
3,45.0,30000.0,3913,Other,2
4,45.0,30000.0,4771,Male,1


### Converting the 3 values back to thier equivalent (NaN) in the `Gender_Encoded` column using the `map` and `lambda` function.

In [64]:
g_data.loc[:, impute_col_encoded] = g_data.loc[:, impute_col_encoded].map(lambda x: np.nan if x == 3 else x)
g_data.head()

C:\Users\CAPACITI-JHB\AppData\Local\Temp\ipykernel_4816\1959509971.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan  0.  1.  2.  1.  1.  0. nan  0.  0.  0.  0.  2. nan  1.  1. nan  1.
 nan  1.  1.  1. nan nan nan nan nan  2.  2.  1. nan  1. nan nan  2.  2.
  0.  2.  0.  0.  0.  0.  0.  1.  2.  0. nan  2.  2.  0.  1.  0.  1. nan
  2. nan  1.  1. nan  1. nan  2. nan  2. nan  1. nan  1. nan nan  0.  1.
  0.  1.  0.  2. nan  1.  1. nan  1.  1.  0.  1.  2.  2. nan  1. nan  2.
  1.  1. nan  2.  1. nan  2. nan  0.  1. nan  0.  1.  2.  1.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  g_data.loc[:, impute_col_encoded] = g_data.loc[:, impute_col_encoded].map(lambda x: np.nan if x == 3 else x)


,Age,Salary,Purchase_Amount,Gender,Gender_Encoded
0,40.0,70000.0,619,NaN,NaN
1,45.0,30000.0,1945,Female,0.0
2,35.0,30000.0,2783,Male,1.0
3,45.0,30000.0,3913,Other,2.0
4,45.0,30000.0,4771,Male,1.0


In [65]:
g_data[impute_col_encoded].value_counts()

Gender_Encoded
1.0    33
0.0    21
2.0    20
Name: count, dtype: int64

## Starting the MICE Imputation

In [66]:
g_data.head()

,Age,Salary,Purchase_Amount,Gender,Gender_Encoded
0,40.0,70000.0,619,NaN,NaN
1,45.0,30000.0,1945,Female,0.0
2,35.0,30000.0,2783,Male,1.0
3,45.0,30000.0,3913,Other,2.0
4,45.0,30000.0,4771,Male,1.0


In [67]:
imputer = IterativeImputer(random_state=100)

Creating a copy of the data we are working on to be able to retrace back.

In [68]:
g_data_train = g_data[['Age', 'Salary', 'Purchase_Amount', 'Gender_Encoded']].copy(deep=True)   #Deep copy of the data
g_data_train.head()

,Age,Salary,Purchase_Amount,Gender_Encoded
0,40.0,70000.0,619,NaN
1,45.0,30000.0,1945,0.0
2,35.0,30000.0,2783,1.0
3,45.0,30000.0,3913,2.0
4,45.0,30000.0,4771,1.0


Fitting the data into the model and transforming it.

In [69]:
imputer.fit(g_data_train)

IterativeImputer(random_state=100)

In [70]:
g_data_imputed = imputer.transform(g_data_train)
g_data_imputed[: 5].round()

array([[4.000e+01, 7.000e+04, 6.190e+02, 1.000e+00],
       [4.500e+01, 3.000e+04, 1.945e+03, 0.000e+00],
       [3.500e+01, 3.000e+04, 2.783e+03, 1.000e+00],
       [4.500e+01, 3.000e+04, 3.913e+03, 2.000e+00],
       [4.500e+01, 3.000e+04, 4.771e+03, 1.000e+00]])

Getting only the values from our `Gender_Encoded` column.

In [71]:
g_data_imputed[:, 3].round()

array([1., 0., 1., 2., 1., 1., 0., 1., 0., 0., 0., 0., 2., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 1., 1., 1., 1., 1.,
       2., 2., 0., 2., 0., 0., 0., 0., 0., 1., 2., 0., 1., 2., 2., 0., 1.,
       0., 1., 1., 2., 1., 1., 1., 1., 1., 1., 2., 1., 2., 1., 1., 1., 1.,
       1., 1., 0., 1., 0., 1., 0., 2., 1., 1., 1., 1., 1., 1., 0., 1., 2.,
       2., 1., 1., 1., 2., 1., 1., 1., 2., 1., 1., 2., 1., 0., 1., 1., 0.,
       1., 2., 1.])

#### Replacing the from the `g_data_imputed` values into our `g_data` dataset.

In [72]:
g_data.loc[:, [impute_col_encoded]] = g_data_imputed[:, 3].round().astype('int')
g_data.head()

,Age,Salary,Purchase_Amount,Gender,Gender_Encoded
0,40.0,70000.0,619,NaN,1.0
1,45.0,30000.0,1945,Female,0.0
2,35.0,30000.0,2783,Male,1.0
3,45.0,30000.0,3913,Other,2.0
4,45.0,30000.0,4771,Male,1.0


#### Inverting our encoded categories back into the original form.

In [73]:
g_data['Gender_Encoded'].unique()

array([1., 0., 2.])

In [74]:
gender_imputed = le.inverse_transform(g_data['Gender_Encoded'].astype('int').round())
gender_imputed[:10]

array(['Male', 'Female', 'Male', 'Other', 'Male', 'Male', 'Female',
       'Male', 'Female', 'Female'], dtype=object)

#### Replace the values in the actual colomn

In [75]:
gender_imputed[10:]

array(['Female', 'Female', 'Other', 'Male', 'Male', 'Male', 'Male',
       'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
       'Male', 'Male', 'Other', 'Other', 'Male', 'Male', 'Male', 'Male',
       'Male', 'Other', 'Other', 'Female', 'Other', 'Female', 'Female',
       'Female', 'Female', 'Female', 'Male', 'Other', 'Female', 'Male',
       'Other', 'Other', 'Female', 'Male', 'Female', 'Male', 'Male',
       'Other', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Other',
       'Male', 'Other', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
       'Female', 'Male', 'Female', 'Male', 'Female', 'Other', 'Male',
       'Male', 'Male', 'Male', 'Male', 'Male', 'Female', 'Male', 'Other',
       'Other', 'Male', 'Male', 'Male', 'Other', 'Male', 'Male', 'Male',
       'Other', 'Male', 'Male', 'Other', 'Male', 'Female', 'Male', 'Male',
       'Female', 'Male', 'Other', 'Male'], dtype=object)

In [76]:
g_data['Gender_Encoded'] = gender_imputed
g_data.head()

C:\Users\CAPACITI-JHB\AppData\Local\Temp\ipykernel_4816\1964302271.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g_data['Gender_Encoded'] = gender_imputed


,Age,Salary,Purchase_Amount,Gender,Gender_Encoded
0,40.0,70000.0,619,NaN,Male
1,45.0,30000.0,1945,Female,Female
2,35.0,30000.0,2783,Male,Male
3,45.0,30000.0,3913,Other,Other
4,45.0,30000.0,4771,Male,Male


In [77]:
g_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Age              88 non-null     float64
 1   Salary           90 non-null     float64
 2   Purchase_Amount  105 non-null    int64  
 3   Gender           74 non-null     object 
 4   Gender_Encoded   105 non-null    object 
dtypes: float64(2), int64(1), object(2)
memory usage: 4.2+ KB


### Null values Imputed in the `Gender` column.

In [78]:
data['Gender'] = g_data['Gender_Encoded']

In [79]:
data['Gender'].unique()

array(['Male', 'Female', 'Other'], dtype=object)

In [80]:
data.head()

,Customer_ID,Age,Salary,City,Gender,Purchase_Amount,Signup_Date,Marital_Status,Education
0,1,40.0,70000.0,New York,Male,619,2022-01-01,Widowed,PhD
1,2,45.0,30000.0,Houston,Female,1945,2022-01-02,Single,PhD
2,3,35.0,30000.0,New York,Male,2783,2022-01-03,NaN,Master
3,4,45.0,30000.0,NaN,Other,3913,2022-01-04,Married,Bachelor
4,5,45.0,30000.0,New York,Male,4771,2022-01-05,Married,NaN
